## Importación de librerias

In [1]:
!pip install sentiment-analysis-spanish
# !pip install gliner
# !pip install sentence_transformers
# !pip install spacy
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentiment_analysis_spanish import sentiment_analysis
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import unicodedata
from unidecode import unidecode
import time
import os

## Clasificacion del estado de animo

In [3]:
def clasificar_estado_animo_con_reglas(frase, resultado=None):
    # Lista de palabras clave para detectar estados negativos
    palabras_negativas = ["llorando", "triste", "deprimido", "desanimado", "solo", "mal", "angustiado", "preocupado", "sufrimiento", "desesperado"]

    # Si detecta palabras negativas, fuerza el estado de ánimo a "Melancólico"
    if any(palabra in frase.lower() for palabra in palabras_negativas):
        return "Melancólico"

    # Verifica si `resultado` es válido antes de intentar acceder a él
    if resultado is not None and len(resultado) > 0:
        label = resultado[0]['label']
        score = resultado[0]['score']

        if label == '5 stars' or (label == '4 stars' and score > 0.8):
            return "Alegre"
        elif label == '3 stars' or (label == '4 stars' and score <= 0.8):
            return "Neutral"
        else:
            return "Melancólico"
    else:
        # Retorna un valor predeterminado si `resultado` es None o vacío
        return "Estado de ánimo desconocido"

In [4]:
sentiment_analyzer = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Ejemplos de uso

In [ ]:
frase_usuario = "ha nacido mi hija"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Alegre


In [ ]:
frase_usuario = "me sient bien"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Neutral


In [ ]:
frase_usuario = "estoy enfermo"
resultado = sentiment_analyzer(frase_usuario)
estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)
print(f"Estado de ánimo clasificado: {estado_animo}")

Estado de ánimo clasificado: Melancólico


## Configuración del modelo

In [5]:
# Cargar el modelo de embeddings
modelo_embeddings = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##Carga de datasets

In [6]:
# Cargar datasets
df_peliculas = pd.read_csv('IMDB-Movie-Data.csv')
df_juegos = pd.read_csv('bgg_database.csv')

In [ ]:
url_libros = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
respuesta = requests.get(url_libros)
soup = BeautifulSoup(respuesta.text, 'html.parser')

# Obtiene el bloque principal de contenido de libros
contenido_div = soup.find("div", class_="container")
bloque_libros = contenido_div.find("div", class_="page_content")
listado_libros = bloque_libros.find('ol')

# Lista para almacenar información de cada libro
libros = []

# Si el bloque de listado de libros es encontrado
if listado_libros:
    # Iteración a través de cada elemento de la lista de libros
    for item in listado_libros.find_all("li"):
        enlace = item.find("a")

        if enlace and 'href' in enlace.attrs:
            # Construye la URL completa del libro
            url_libro = 'https://www.gutenberg.org' + enlace['href']

            # Solicita la página del libro específico
            respuesta_libro = requests.get(url_libro)
            soup_libro = BeautifulSoup(respuesta_libro.text, 'html.parser')

            # Extrae la información relevante de la página del libro
            div_principal = soup_libro.find("div", class_="container")
            contenido_pagina = div_principal.find("div", class_="page_content", id='content')
            cuerpo_pagina = contenido_pagina.find("div", class_="page-body")
            contenedor_tabs = cuerpo_pagina.find("div", id="tabs-wrapper")
            registro_bibliografico = contenedor_tabs.find("div", id="bibrec")
            ebook_info = registro_bibliografico.find("div", {"typeof": "pgterms:ebook"})
            tabla_info = ebook_info.find("table", class_="bibrec")

            # Diccionario para almacenar la información del libro
            datos_libro = {}
            temas = []

            # Itera sobre cada fila de la tabla de información
            for fila in tabla_info.find_all("tr"):
                clave = fila.find("th")
                valor = fila.find("td")

                if clave and valor:
                    etiqueta = clave.text.strip()
                    contenido = valor.text.strip()

                    # Almacena los datos relevantes
                    if etiqueta == 'Author':
                        datos_libro['Autor'] = contenido
                    elif etiqueta == 'Title':
                        datos_libro['Titulo'] = contenido
                    elif etiqueta == 'Summary':
                        datos_libro['Resumen'] = contenido
                    elif etiqueta == 'Subject':
                        temas.append(contenido)  # Agrega cada tema a la lista de temas

            # Almacena la lista de temas en el diccionario de datos del libro
            datos_libro['Temas'] = temas
            libros.append(datos_libro)

            # Pausa para evitar sobrecargar el servidor
            time.sleep(1)


In [ ]:
df_libros = pd.DataFrame(libros)
# Guarda el DataFrame en un archivo CSV con "#" como separador para facilitar la manipulación
df_libros.to_csv('libros.csv', index=False, sep='#')

# Descarga el archivo para evitar tener que ejecutar el código cada vez que se necesiten los datos
from google.colab import files
files.download("libros.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

En caso de que ya se tenga el archivo libros.csv, se ejecuta este codigo.

In [10]:
df_libros = pd.read_csv('libros.csv', sep='#')

Se verifica que haya nulos. Si es que los hay, se elimina la fila entera en la que hay un valor nulo, ya que no es necesario mantener toda la fila.


In [11]:
df_libros.isna().sum()
#eliminacion de nulos
df_libros.dropna(inplace=True)

In [18]:
df_libros_reducido = df_libros.sample(n=300, random_state=1).reset_index(drop=True)

## Embeddings

Para comparar la preferencia del usuario con los títulos de las recomendaciones, usaremos la similitud de coseno, optimizando su cálculo al estructurar los datos para mejorar la precisión y eficiencia.

In [12]:
# Modelo de embeddings
modelo_embeddings = SentenceTransformer('all-mpnet-base-v2')

# Función para guardar los vectores embeddings en un archivo .pkl
def guardar_embeddings(titulos, filename):
    vectores = [modelo_embeddings.encode(titulo) for titulo in titulos]
    with open(filename, 'wb') as f:
        pickle.dump(vectores, f)

# Función para cargar los vectores embeddings desde un archivo .pkl
def cargar_embeddings(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def generar_o_cargar_embeddings(df, columnas, filename):
    if not os.path.exists(filename):
        # Combinar valores de las columnas especificadas
        titulos = df[columnas].astype(str).agg(' '.join, axis=1).tolist()
        guardar_embeddings(titulos, filename)
    return cargar_embeddings(filename)


# Cargar o crear embeddings para películas y juegos
vector_peliculas = generar_o_cargar_embeddings(df_peliculas, ['Title','Genre', 'Description', 'Director' ], 'vector_peliculas.pkl')
vector_juegos = generar_o_cargar_embeddings(df_juegos, ['game_name', 'categories'], 'vector_juegos.pkl')
vector_libros = generar_o_cargar_embeddings(df_libros, ['Author', 'Title', 'Summary', 'Subjects'], 'vector_libros.pkl')

#ver error aca arriba


# Cálculo y almacenamiento de embeddings de libros solo si no están calculados
if 'Texto_completo_embedding' not in df_libros.columns:
    df_libros['Texto_completo'] = df_libros['Summary'].fillna('').apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)) + " " + \
                                  df_libros['Subjects'].fillna('').apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

    df_libros['Texto_completo_embedding'] = df_libros['Texto_completo'].apply(lambda x: modelo_embeddings.encode(x))



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
print("Tamaño de vector_peliculas:", len(vector_peliculas))
print("Tamaño de vector_juegos:", len(vector_juegos))
print("Tamaño de vector_libros:", len(vector_libros))

Tamaño de vector_peliculas: 1000
Tamaño de vector_juegos: 1000
Tamaño de vector_libros: 773


In [24]:
def identificar_tipo_contenido(frase_usuario):
    """
    Determina el tipo de contenido (película, libro, juego de mesa) según palabras clave en la frase.
    """
    frase_sin_acentos = unidecode(frase_usuario.lower().strip())
    if "pelicula" in frase_sin_acentos:
        return "pelicula"
    elif "libro" in frase_sin_acentos:
        return "libro"
    elif "juego de mesa" in frase_sin_acentos:
        return "juego"
    return None


In [41]:
def buscar_opciones_tfidf(df, frase, tipo):
    # Concatenar los campos de interés según el tipo de contenido
    if tipo == "pelicula":
        df['Texto_completo'] = (
            df['Title'].fillna('') + " " +
            df.get('Description', '').fillna('') + " " +
            df.get('Genre', '').fillna('')
        )
    elif tipo == "libro":
        df['Texto_completo'] = (
            df.get('Summary', '').fillna('') + " " +
            df.get('Subjects', '').apply(lambda x: ' '.join(x) if isinstance(x, list) else x).fillna('')
        )
    elif tipo == "juego":
        df['Texto_completo'] = (
            df['game_name'].fillna('') + " " +
            df.get('categories', '').apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x)).fillna('')
        )

    # Crear el vectorizador TF-IDF y ajustar el DataFrame
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Texto_completo'])

    # Transformar la frase ingresada en un vector TF-IDF
    frase_vector = tfidf_vectorizer.transform([frase])

    # Calcular la similitud del coseno entre la frase y el DataFrame
    similarity_matrix = cosine_similarity(frase_vector, tfidf_matrix)

    # Devolver las 5 opciones más similares
    df['Similaridad'] = similarity_matrix.flatten()
    return df.nlargest(5, 'Similaridad')


In [44]:
# Flujo principal
def flujo_principal():
    frase_usuario = input("¿Cómo te sientes hoy? ")
    resultado = sentiment_analyzer(frase_usuario)  # Analizar la frase del usuario
    estado_animo = clasificar_estado_animo_con_reglas(frase_usuario, resultado)  # Clasificar el estado de ánimo
    print(f"Estado de ánimo clasificado: {estado_animo}")

    # Ingreso de Preferencias
    frase_usuario = input("Ingresa una frase que describa la temática que te gustaría explorar: ")
    tipo_contenido = identificar_tipo_contenido(frase_usuario)

    # Búsqueda de Opciones
    if tipo_contenido == "pelicula":
        opciones = buscar_opciones_tfidf(df_peliculas, frase_usuario, tipo_contenido)  # Si usas embeddings
        print("\nOpciones de Películas:", opciones[['Title', 'Description', 'Genre', 'Similaridad']])
    elif tipo_contenido == "libro":
        opciones = buscar_opciones_tfidf(df_libros, frase_usuario, tipo_contenido)  # Cambiar a la nueva función
        print("\nOpciones de Libros:", opciones[['Title', 'Summary', 'Subjects']])
    elif tipo_contenido == "juego":
        opciones = buscar_opciones_tfidf(df_juegos, frase_usuario, tipo_contenido)  # Si usas embeddings
        print("\nOpciones de Juegos de Mesa:", opciones[['game_name', 'categories']])
    else:
        print("No se reconoció el tipo de contenido. Por favor, incluye 'película', 'libro' o 'juego de mesa' en tu frase.")

# Llamar al flujo principal
if __name__ == "__main__":
    flujo_principal()

¿Cómo te sientes hoy? alegre
Estado de ánimo clasificado: Alegre
Ingresa una frase que describa la temática que te gustaría explorar: pelicula de animacion

Opciones de Películas:                        Title  \
793      Ma vie de Courgette   
883                      Rio   
742   El secreto de sus ojos   
0    Guardians of the Galaxy   
1                 Prometheus   

                                           Description  \
793  After losing his mother, a young boy is sent t...   
883  When Blu, a domesticated macaw from small-town...   
742  A retired legal counselor writes a novel hopin...   
0    A group of intergalactic criminals are forced ...   
1    Following clues to the origin of mankind, a te...   

                          Genre  Similaridad  
793      Animation,Comedy,Drama     0.240444  
883  Animation,Adventure,Comedy     0.212964  
742       Drama,Mystery,Romance     0.188923  
0       Action,Adventure,Sci-Fi     0.000000  
1      Adventure,Mystery,Sci-Fi     0.00000